# Progressive Sampling
The general idea is to divide the data into bins of equal density. 
Then, we progressively maintain a representative set of points for each bin, to be able to capture the distributions of the entire dataset whenever we recompute the doi function or train the classifier.

This task is split into the following subtasks: 
1. compute the doi function over all items across all bins
2. rebin the data based on their doi values
3. update items in bins that have changed "too much"

## 1) Compute the doi function
For simplicity, we will use outlier detection here, which for every item computes, whether or not it is an outlier in the dataset, given an approximate "contamination" of the data.

In [2]:
from sklearn.ensemble import IsolationForest
doi = IsolationForest(contamination=0.1, random_state=0)

## 2) Binning
Given a distribution of floating point values, progressively maintain bins of equal density for them.

In [37]:
import numpy as np
from random import random
from sklearn.datasets import make_blobs
from sklearn.preprocessing import KBinsDiscretizer
import matplotlib.pyplot as plt

N = 10000
training_size = 100
features = 2
chunk_size = 1000

blobs_params = dict(n_samples=N, n_features=features)
X = make_blobs(centers=6, cluster_std=1, **blobs_params)[0]
y = doi.fit_predict(X)

n_bins = 5
binning = KBinsDiscretizer(n_bins=n_bins, encode="ordinal", strategy="uniform")
edges = binning.fit(y.reshape(-1, 1)).bin_edges_[0]

previous_bins = []

distributions = [np.random.normal(random(), random() * 0.25, N) for _ in range(N//chunk_size)]
for i, distribution in enumerate(distributions):
  y_ = y + distribution
  binning.fit(y_.reshape(-1, 1))
  
  fig, (ax1, ax2, ax3) = plt.subplots(3, 1)
  bins = binning.transform(y_.reshape(-1, 1))
  if i > 0:
    delta_bins = bins - previous_bins
    ax3.hist(delta_bins, fc="green", bins=edges, **hist_params)

  previous_bins = bins

  bins = bins.flatten().astype(int)
  bins = np.array([edges[bins[i]] for i in range(len(bins))])
  
  hist_params = {"alpha": .7, "ec": "#000"}
  ax1.set_title("original data")
  ax1.hist(y_, **hist_params)
  ax2.hist(bins, fc="orange", bins=edges, **hist_params)
  

  previous_bins = bins

## 3) Sampling
A good candidate seems to be so-called reservoir sampling, which progressively updates a representative sample of the data seen so far, thus at every point maintainnig a sample where each point may have been picked with probability := 1/(number of points processed so far).

See [https://dl.acm.org/doi/pdf/10.1145/3147.3165](https://dl.acm.org/doi/pdf/10.1145/3147.3165)

### Naive chunk-based Reservoir sampling
Pseudo code:
```
# we progress over a dataset X of size N and want to maintain a progressive random sample S of 
# size n.
# Every item x_t from the dataset is numbered based on the timepoint t at which it is 
# processed: x_t with t \in [0 ... N]. We go through the data in linear order.
# for x_t in X:
  # if t <= n, then x_t is put into S.
  # otherwise, x_t becomes a candidate that may replace an item in S:
  #    generate a random integer r from the interval [0, t]. If r < n, then replace the item S[r] 
  #    with with x_t
# the probability of an item becoming part of the sample S is n/t and is decreasing with t.
# the probability of being replaced is higher, the longer an item is part of Sy.
```

In [111]:
from sklearn.utils.random import sample_without_replacement
from random import randint
import numpy as np

reservoir = np.array([])
k = 5

X = np.arange(0, 1000)
chunk_size = 10

for i in range(0, len(X) // chunk_size):
  chunk = X[i * chunk_size:(i+1) * chunk_size]

  if i == 0:
    sample = sample_without_replacement(n_population=chunk_size, n_samples=k, random_state=i)
    reservoir = chunk[sample]
  else:
    candidates = np.array([randint(0, i*chunk_size) for _ in range(chunk_size)])
    picks = candidates[candidates < k]
    reservoir[picks] = chunk[picks]

reservoir

array([510, 301, 192, 573, 944])

### Optimized chunk-based Reservoir Sampling

Quote from the reservoir sampling paper:
> The random variable S(n, t) is defined to be the number of records in the file that are skipped over before the next record is chosen for the reservoir, where n is the size of the sample and where t is the number of records processed so far. For notational simplicity, we shall often write 9 for S(n, t),in which case the parameters n and t will be implicit.

In [110]:
from sklearn.utils.random import sample_without_replacement
from random import randint
import numpy as np

reservoir = np.array([])
k = 5

X = np.arange(0, 1000)
chunk_size = 10

skip = k

for i in range(0, len(X) // chunk_size):
  chunk = X[i * chunk_size:(i+1) * chunk_size]

  if i == 0:
    sample = sample_without_replacement(n_population=chunk_size, n_samples=k, random_state=i)
    reservoir = chunk[sample]
    skip = randint(k, (i+1)*chunk_size)
  elif skip < i*chunk_size:
    candidate = chunk[skip % chunk_size]
    reservoir[randint(0, k-1)] = candidate
    skip = randint(k, i*chunk_size*2)

reservoir

array([262, 104, 972, 996, 984])